In [1]:
import glob
import os
import sys
import datetime
import numpy as np
import pandas as pd

def find_consecutive_ranges(lst):
    if not lst:
        return []
    
    ranges = []
    start = lst[0]
    
    for i in range(1, len(lst)):
        if lst[i] != lst[i - 1] + 1:
            ranges.append([start, lst[i - 1]])
            start = lst[i]
    
    ranges.append([start, lst[-1]])  # Append the last range
    
    return ranges

In [19]:
output_IM_folder_path = "./optical/"
report_path = "./report/"
os.makedirs(output_IM_folder_path,exist_ok=True)
os.makedirs(report_path,exist_ok=True)
files = glob.glob('./raw_data/*.bin')
files.sort()
file_name = files[-3]
reports = glob.glob('./report/*.csv')
reports.sort()

# read the raw data, split the data into packets using sync bytes
with open(file_name, 'rb') as f:
    mpduPackets = f.read().split(b'\x1A\xCF\xFC\x1D')[1:]

if sys.argv == "detail":
    # output file that contains the header information of the packets
    # fout_name = f'./DQ_check/{'/'.join(file_name.split('/')[2:])}'
    fout_name = f'./{file_name.split('/')[-1]}'
    fout_name = fout_name.replace('.bin', '_header.txt')
    print(f'Detail output: {fout_name}')
    fout = open(fout_name, 'w')
else:
    dt_now = datetime.datetime.now()
    time_now = dt_now.strftime('%d_%H%M%S')
    print(f'Raw data file: {file_name}')
    if len(reports) == 0:
        print('No report file found, create a new one.')
        fout_name = f'{report_path}report_0000_{time_now}.csv'
        with open(fout_name, 'w') as f:
            f.write('Filename,Type,Start_Packet_number,End_Packet_number,Incompleteness\n')
    else:
        fout_name = reports[-1]
        if os.path.getsize(fout_name) > 10000:
            print('The last report file is too large, create a new one.')
            fout_name = f'{report_path}report_{str(len(reports)).zfill(4)}_{time_now}.csv'
            with open(fout_name, 'w') as f:
                f.write('Filename,Type,Start_Packet_number,End_Packet_number,Incompleteness\n')
        else:
            print(f'Write to the last report file: {fout_name}')

Raw data file: ./raw_data/F20250109130351.bin
Write to the last report file: ./report/report_0000_05_143846.csv


In [20]:
imgData = bytes()
headers = [[], [], [], []]
hk = []

# loop through all the packets
if sys.argv == "detail":
    for k, packet in enumerate(mpduPackets):
        # classify the packets by the VCDU header
        if packet[28:30] == b'\x55\x40':
            headers[0].append('IM')
        elif packet[28:30] == b'\x40\x3F':
            headers[0].append('HK')
            hk.append(packet.hex())
        else:
            headers[0].append('UnClassified')
        
        headers[1].append(int.from_bytes(packet[30:33], 'big'))
        headers[2].append(packet[34])
        headers[3].append(packet[1])
        fout.write(
            f'{headers[0][-1]}, {headers[1][-1]}, {headers[2][-1]}, {headers[3][-1]}\n'
        )
    fout.write(f'Number of Packets: {len(mpduPackets)}\n')
    fout.close()
else:
    for k, packet in enumerate(mpduPackets):
        # classify the packets by the VCDU header
        if packet[28:30] == b'\x55\x40':
            headers[0].append('IM')
            imgData += packet[56:-160]
        elif packet[28:30] == b'\x40\x3F':
            headers[0].append('HK')
            hk.append(packet.hex())
        else:
            headers[0].append('UnClassified')
        
        headers[1].append(int.from_bytes(packet[30:33], 'big'))
        headers[2].append(packet[34])
        headers[3].append(packet[1])
        # *** save hk data somewhere ***

In [23]:
try:
    headerDF = pd.DataFrame(np.array(headers).T, columns=['VCDU', 'PSC', 'IB', 'DQ'])
    IM, HK = headerDF[headerDF['VCDU'] == 'IM']['PSC'].astype(int), headerDF[headerDF['VCDU'] == 'HK']['PSC'].astype(int)
    bad_IM, bad_HK = headerDF[headerDF['DQ'] != '0']['PSC'].astype(int), headerDF[headerDF['DQ'] != '0']['PSC'].astype(int)
    IM_range = set(range(0, 16620))
    max_HK = HK[HK < 10000].max()
    HK_range = set(range(int(min(HK)), int(max_HK)+1))
    missing_IM = IM_range - set(IM)
    missing_HK = HK_range - set(HK)
    missing_IM = sorted(list(missing_IM))
    missing_HK = sorted(list(missing_HK))
    # request_IM = set(IM) | set(bad_IM)
    # request_HK = set(HK) | set(bad_HK)
    missing_rate_IM = (len(missing_IM)/16621)*100
    missing_segment_IM = find_consecutive_ranges(list(missing_IM))
    missing_segment_HK = find_consecutive_ranges(list(missing_HK))
    
    if sys.argv == "detail":
        fout = open(fout_name, 'a')
        fout.write(f'Total Image Packets: {headerDF[headerDF['VCDU'] == 'IM'].shape[0]}\n')
        fout.write(f'UnClassified Packets: {headerDF[headerDF['VCDU'] == 'UnClassified'].shape[0]}\n')
        fout.write(f'Sequence number of Missing Image Packets: {missing_IM}\n')
        fout.write(f'Number of Missing Image Packets: {len(missing_IM)}\n')
        fout.write(f'Sequence number of Missing HK Packets: {missing_HK}\n')
        fout.write(f'Number of bad quality Image Packets: {len(bad_IM)}\n')
        fout.write(f'Number of Missing HK Packets: {len(missing_HK)}\n')
        fout.write(f'Number of bad quality HK Packets: {len(bad_HK)}\n')
        fout.write(f'Total missing packets: {len(missing_IM)+len(missing_HK)}\n')
        fout.write(f'Segment of request image packets: {missing_segment_IM}\n')
        fout.write(f'Segment of request HK packets: {missing_segment_HK}\n')
        fout.write(f'Request image packets rate (missing image/Image packet %): {missing_rate_IM}\n')
        fout.close()
    else:
        nfiles = len(glob.glob(output_IM_folder_path+'*.bin'))
        nfiles = str(nfiles).zfill(4)
        # only save the image data if there is no missing image packets
        if missing_rate_IM == 0:
            # no missing packets, save the image data
            imgData = imgData.rstrip(b'\0')
            with open(f'./optical/opt_frame_{nfiles}_{file_name.split('/')[-1]}', 'wb') as f:
                f.write(imgData)
        else:
            # output the report for the missing packets
            with open(fout_name, 'a') as f:
                for segment in missing_segment_IM:
                    f.write(f'{file_name.split('/')[-1]},IM,{segment[0]},{segment[1]},{missing_rate_IM}\n')
                for segment in missing_segment_HK:
                    f.write(f'{file_name.split('/')[-1]},HK,{segment[0]},{segment[1]},-1\n')

except Exception as e:
    with open(fout_name, 'a') as f:
        f.write(f'{file_name.split('/')[-1]},Error,-1,-1,-1\n')
    print(f'Error in {file_name}: {e}')